In [1]:
import pandas as pd

## Apriori算法

In [52]:
# 自定义连接函数，用于实现L_{k-1}到C_k的连接
def connect_string(x, ms):
    # split 返回结果为list -> 'a' => ['a']
    x = list(map(lambda i: sorted(i.split(ms)), x))
    print(x, '____1')
    l = len(x[0])
    r = []
    print(x[1][:0], '____2')
    print(sorted([x[l-2][l-1], x[0][l-1]]), '___3')
    for i in range(len(x)):
        for j in range(i, len(x)):
            if x[i][:l-1] == x[j][:l-1] and x[i][l-1] != x[j][l-1]:
                r.append(x[i][:l-1] + sorted([x[j][l-1], x[i][l-1]]))
    return r

# 寻找关联规则的函数
def find_rule(d, support, confidence, ms = u'-->'):
    # 定义输出结果
    result = pd.DataFrame(index = ['support', 'confidence'])
    # 支持度序列 平均数(每一项的和(列) / 长度(列)) => 项集的频率
    support_series = 1.0 * d.sum() / len(d)
    # 初步根据支持度筛序, 取索引值('a', 'b, 'c'...)
    print(list(support_series[support_series > support]), '___4')
    column = list(support_series[support_series > support].index)
    print(column, '___5')
    
    k = 0
#     while len(column) > 1:
    k += 1
    print('\n正在进行第{}次搜索...'.format(k))
    column = connect_string(column, ms)
    print(column, '___6')
    print('数目: %s' % len(column))
    
    # 新一批支持度的计算函数
    # 按列(竖着看)计算乘积
    sf = lambda i: d[i].prod(axis = 1, numeric_only = True)
    # 创建连接数据（转置与原数据维度保持一致），这一操作很耗时间，耗内存，当数据集较大时，可以考虑并行计算进行优化
    d_2 = pd.DataFrame(list(map(sf, column)), index = [ms.join(i) for i in column]).T
    print(d_2, '___7')
    print([ms.join(i) for i in column], '___8')
    # 计算连接后的支持度 -> 平均数(每一项的和(列) / 长度(列)) => 项集的频率
    support_series_2 = 1.0 * d_2[[ms.join(i) for i in column]].sum() / len(d)
    print(type(support_series_2), '___9')
    # 新一轮支持度筛序,去索引值
    column = list(support_series_2[support_series_2 > support].index)
    print(column, '__10')
    # 合并支持度序列, Series需要赋值操作
    support_series = support_series.append(support_series_2)
    print(support_series, '__11')
    column2 = []
    
    # 

In [22]:
# 菜品订单数据
inputfile = 'E:\\NLP\\Python数据分析与挖掘实战\\chapter5\\demo\\data\\menu_orders.xls'
outputfile  = 'E:\\NLP\\Python数据分析与挖掘实战\\chapter5\\demo\\tmp\\menu_orders_fit_end.xls'
data = pd.read_excel(inputfile, header = None)
print(data.as_matrix())

[['a' 'c' 'e' nan]
 ['b' 'd' nan nan]
 ['b' 'c' nan nan]
 ['a' 'b' 'c' 'd']
 ['a' 'b' nan nan]
 ['b' 'c' nan nan]
 ['a' 'b' nan nan]
 ['a' 'b' 'c' 'e']
 ['a' 'b' 'c' nan]
 ['a' 'c' 'e' nan]]


In [23]:
print(u'\n转换原始数据至0-1矩阵')
# 转换0-1矩阵的过渡函数
# pd.notnull -> 参考python 各种实验_02 => 取非空的值
ct = lambda x: pd.Series(1, index = x[pd.notnull(x)])
# 用map方式执行
b = map(ct, data.as_matrix())
# 实现矩阵转换，空值用0填充
data = pd.DataFrame(list(b)).fillna(0)
print(data)
# print(data.sum())
# print(len(data))
print(1.0 * data.sum() / len(data))
# 转换完毕后删除中间变量 b，节省内存
del b


转换原始数据至0-1矩阵
     a    b    c    d    e
0  1.0  0.0  1.0  0.0  1.0
1  0.0  1.0  0.0  1.0  0.0
2  0.0  1.0  1.0  0.0  0.0
3  1.0  1.0  1.0  1.0  0.0
4  1.0  1.0  0.0  0.0  0.0
5  0.0  1.0  1.0  0.0  0.0
6  1.0  1.0  0.0  0.0  0.0
7  1.0  1.0  1.0  0.0  1.0
8  1.0  1.0  1.0  0.0  0.0
9  1.0  0.0  1.0  0.0  1.0
a    0.7
b    0.8
c    0.7
d    0.2
e    0.3
dtype: float64


In [4]:
# 最小支持度
support = 0.2
# 最小置信度
confidence = 0.5
# 连接符，用来区分不同的元素，如 A -> B(需要保证原始表格中不含有该字符)
ms = '-->'

In [53]:
# 保存结果
find_rule(data, support, confidence, ms)

[0.69999999999999996, 0.80000000000000004, 0.69999999999999996, 0.29999999999999999] ___4
['a', 'b', 'c', 'e'] ___5

正在进行第1次搜索...
[['a'], ['b'], ['c'], ['e']] ____1
[] ____2
['a', 'e'] ___3
[['a', 'b'], ['a', 'c'], ['a', 'e'], ['b', 'c'], ['b', 'e'], ['c', 'e']] ___6
数目: 6
   a-->b  a-->c  a-->e  b-->c  b-->e  c-->e
0    0.0    1.0    1.0    0.0    0.0    1.0
1    0.0    0.0    0.0    0.0    0.0    0.0
2    0.0    0.0    0.0    1.0    0.0    0.0
3    1.0    1.0    0.0    1.0    0.0    0.0
4    1.0    0.0    0.0    0.0    0.0    0.0
5    0.0    0.0    0.0    1.0    0.0    0.0
6    1.0    0.0    0.0    0.0    0.0    0.0
7    1.0    1.0    1.0    1.0    1.0    1.0
8    1.0    1.0    0.0    1.0    0.0    0.0
9    0.0    1.0    1.0    0.0    0.0    1.0 ___7
['a-->b', 'a-->c', 'a-->e', 'b-->c', 'b-->e', 'c-->e'] ___8
<class 'pandas.core.series.Series'> ___9
['a-->b', 'a-->c', 'a-->e', 'b-->c', 'c-->e'] __10
a        0.7
b        0.8
c        0.7
d        0.2
e        0.3
a-->b    0.5
a-->c  